# 양자화 모델 파인 튜닝

## LLM 양자화에 필요한 패키지 설치

In [1]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import GPTNeoForCausalLM, GPTNeoConfig, AdamW, get_linear_schedule_with_warmup
from transformers import GPT2Tokenizer, GPT2TokenizerFast
from torch.utils.data import Dataset, DataLoader
from transformers import AutoConfig
import torch
import pandas as pd
import accelerate

## 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Backbone model로 경량화 모델 로드하기 - 양자화

In [4]:
model_id = "kyujinpy/Ko-PlatYi-6B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [5]:
# 모델 살펴보기
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(78464, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (n

## Backbone model이 잘 실행되는지 확인

In [7]:
device = "cuda:0"

messages = [
    {"role": "user",
     "content": "일본 여행에 대한 블로그를 작성해줘.블로그 내용에는 맛집, 유명 관광지, 여행지의 랜드마크 등을 꼭 언급해야하고 500자 이상으로 작성해줘"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<|startoftext|> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

일본 여행에 대한 블로그를 작성해줘.블로그 내용에는 맛집, 유명 관광지, 여행지의 랜드마크 등을 꼭 언급해야하고 500자 이상으로 작성해줘 [/INST]<|endoftext|>


## Fine Tuning 데이터 로드 및 전처리

In [21]:
import pandas as pd

# 데이터셋을 불러오는 함수
def load_dataset(file_path):
    data = pd.read_csv(file_path, encoding='utf-8')
    return data # '본문' 대신 '내용'을 사용


# Fine-tuning dataset
train_data = load_dataset('/content/blog_crawling_new.csv')

In [22]:
train_data.head()
train_dt = train_data.drop(columns = 'Unnamed: 0')

In [27]:
df_train = train_dt.drop(0)
df_train.head()

,주제,제목,링크,내용
1,여행,터키 일주일 자유 여행 소울투어터키 패키지 솔직 후기,https://blog.naver.com/daun1217/223343210713,​ ​직장인의 일주일 터키 자유 여행굉장히 한정된 시간속에서 넓디 넓은튀르키예라는 ...
2,여행,대만 여행지원금 사용 대만 딤섬 맛집 딤딤섬 타이베이,https://blog.naver.com/wlsgml850/223343540698,대만 여행지원금 사용처 T A I P E I 이번에 다녀온 대만여행 첫날 일정에...
3,여행,세부 여행 블루워터 마리바고 리조트 룸 조식 수영장,https://blog.naver.com/charla1005/223330883703,세부 여행 블루워터 마리바고 리조트에서 3박을 했어요~ 3박5일의 여행이었는데 막...
4,여행,필리핀 보라카이 여행 경비 3박5일 준비 주의사항 A to Z 꿀팁,https://blog.naver.com/uni7741/223343548659,​ 필리핀 보라카이 여행 경비 3박5일 준비주의사항 A to Z 꿀팁글·사진 by ...
5,여행,중국 장가계 패키지 여행 천문산 케이블카 유리잔도,https://blog.naver.com/weeeunjee/223342973626,​중국 장가계 패키지 여행 천문산 케이블카 유리잔도 오늘은 일전에 하나투어로 다...


In [38]:
df_train['내용'] = df_train['내용'].str.strip()
df = df_train

In [41]:
import json

# JSON 데이터 생성
json_data = []
for index, row in df.iterrows():
    item = {
        "input": row["주제"] + " " + row["제목"],
        "output": row["내용"],
        "instruction": "당신은 인공지능 블로그 작성자입니다. 사용자가 여러분에게 주제를 줍니다. 당신의 목표는 주어진 주제를 바탕으로 블로그 내용을 작성하는 것입니다. 블로그를 작성하는 동안 다양한 생각과 여러가지 방식을 고려하여 500자 이상의 블로그를 작성하세요 ."
    }
    json_data.append(item)

# JSON 파일로 저장
with open("blog_data.json", "w", encoding="utf-8") as outfile:
    json.dump(json_data, outfile, indent=4, ensure_ascii=False)

print("JSON 파일이 생성되었습니다.")

JSON 파일이 생성되었습니다.


**2024 02 12 파인 튜닝 데이터 전처리 완료, Koalpaca 모델 선택 완료, 벤치 마킹 코드 선택 완료**

## Backbone model을 peft로 wrapping

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, AutoConfig
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
from peft import LoraConfig, TaskType
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
# peft로 fine tuning 경량화
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [ ]:
from peft import get_peft_model

# backcone model을 peft로 wrapping
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# train parameter 수를 많이 줄임
# 6,182,801,408 -> 3,276,800 약 0.05%의 파라미터만 update 하면 됨

trainable params: 3,276,800 || all params: 6,182,801,408 || trainable%: 0.05299862932294914


## Final model config 설정 및 Fine tuning 진행

In [ ]:
# Model configuration
# backbone model config 설정
config = AutoConfig.from_pretrained(model_id)
config.num_train_epochs = 3
config.per_device_train_batch_size = 2
config.gradient_accumulation_steps = 1
config.learning_rate = 5e-5
config.logging_steps = 10
config.save_steps = 100
config.warmup_steps = 100
config.weight_decay = 0.01
config.load_in_8bit = True
config.fp16 = True
config.output_attentions = False
config.output_hidden_states = False
config.return_dict_in_generate = True

In [ ]:
# Move the model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Initialize the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=config.learning_rate, eps=1e-8, no_deprecation_warning=True)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup_steps, num_training_steps=config.num_train_epochs * len(train_dataloader))

In [ ]:
# Training loop
for epoch in range(config.num_train_epochs):
    model.train()
    for step, batch in enumerate(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = input_ids.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # 위 코드에서 TypeError: LlamaForCausalLM.forward() got an unexpected keyword argument 'decoder_input_ids' 이런 오류 발생
        # 어케 해결할지 모르겠음
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()

        if config.logging_steps > 0 and (step + 1) % config.logging_steps == 0:
            print(f'Epoch: {epoch + 1}/{config.num_train_epochs}, Step: {step + 1}/{len(train_dataloader)}, Loss: {loss.item()}')

    # Save the model
    if (epoch + 1) % config.save_steps == 0:
        model.save_pretrained(f'path/to/save/model_epoch_{epoch + 1}')